In [ ]:
import tensorflow
from tensorflow.keras.applications import DenseNet201

In [ ]:
from tensorflow.keras import Model,Input
from tensorflow.keras import Sequential
from tensorflow.keras.layers import *
model = Sequential()

base_model = DenseNet201(include_top=False,weights='imagenet')#,input_shape=(32,32,3))
#base_model.trainable = False

inputLayer = Input(shape=(32,32,3))

#x = base_model(inputLayer,training=False)
for layer in base_model.layers:
    if 'conv5' in layer.name:
        layer.trainable = True
    else:
        layer.trainable = False
#outputLayer = Dense(10,activation='softmax')
model.add(inputLayer)
model.add(base_model)
model.add(Flatten())
model.add(BatchNormalization())

model.add(Dense(256,activation='relu'))
model.add(Dropout(0.4))
model.add(BatchNormalization())

model.add(Dense(128,activation='relu'))
model.add(Dropout(0.4))

model.add(Dense(10,activation='softmax'))


#model = Model(inputLayer,outputLayer)
print(model.summary())

In [ ]:
!pip install py7zr
!python -m py7zr x ../input/cifar-10/train.7z #takes 10 oof minutes of CPU to decompress this file. Big OOF


In [ ]:
import pandas as pd
import glob
import numpy as np
from PIL import Image
y = pd.read_csv('../input/cifar-10/trainLabels.csv')
filelist = glob.glob("../working/train/*")
filelist = sorted(filelist,key= lambda x:int(x.split("/")[-1].split(".")[0]))
X = np.array([np.array(Image.open(fname)) for fname in filelist])
print(f"X.shape : {X.shape} y.shape : {y.shape}")

In [ ]:
# got X and y at this point, now some preprocessing
import tensorflow
y = y['label'].values

X = X.astype('float32')

X = X/255.0

temp = y
categories = list(pd.CategoricalIndex(temp).categories)
mapping = dict()
for i in range(len(categories)):
    mapping[categories[i]] = i
for i in range(len(temp)):
    temp[i] = mapping[temp[i]]
y = tensorflow.keras.utils.to_categorical(temp)

In [ ]:
# data augmentation
# starting Data Augmentation now

from tensorflow.keras.preprocessing.image import ImageDataGenerator
datagen = ImageDataGenerator(
            rotation_range=10,
            zoom_range=0.1,
            width_shift_range=0.1,
            height_shift_range=0.1
            )
datagen.fit(X)

In [ ]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [ ]:
from sklearn.model_selection import train_test_split

X_train,X_val,y_train,y_val = train_test_split(X,y,test_size=0.2,random_state=1)
model.fit(datagen.flow(X_train,y_train),batch_size=128,epochs=10,verbose=1,validation_data=(X_val,y_val))

In [ ]:
model.fit(datagen.flow(X,y),batch_size=128,epochs=20,verbose=1)